In [30]:
import os

#Data Encryption Libraries
from cryptography.hazmat.primitives import padding as p
from cryptography.hazmat.primitives.asymmetric import rsa, padding as asp
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers.modes import CBC
from cryptography.hazmat.primitives.hashes import SHA256

In [31]:
def hybrid_rsa_aes_encryption(packet, public_key):

    pkcs7_padder = p.PKCS7(AES.block_size).padder()
    padded_packet = pkcs7_padder.update(packet) + pkcs7_padder.finalize()

    #AES 256 random key generation
    key = os.urandom(256 // 8)

    #Corresponding IV generation
    iv = os.urandom(128 // 8)

    #AEC/CBC cipher
    aes_cbc_cipher = Cipher(AES(key), CBC(iv))

    #encrypting padded packets
    cipher_packet = aes_cbc_cipher.encryptor().update(padded_packet)

    #encrypting keys
    oaep_padding = asp.OAEP(mgf=asp.MGF1(algorithm=SHA256()), algorithm=SHA256(), label=None)
    cipher_key = public_key.encrypt(key, oaep_padding)

    return {'iv': iv, 'cipher_packet': cipher_packet}, cipher_key

In [32]:
def hybrid_rsa_aes_decryption(cipher_packet, cipher_key, private_key):

    #AES decryption
    oaep_padding = asp.OAEP(mgf=asp.MGF1(algorithm=SHA256()), algorithm=SHA256(), label=None)
    recovered_key = private_key.decrypt(cipher_key, oaep_padding)

    #decrypt padded packet
    aes_cbc_cipher = Cipher(AES(recovered_key), CBC(cipher_packet['iv']))
    recovered_padded_packet = aes_cbc_cipher.decryptor().update(cipher_packet['cipher_packet'])

    #removing packet padding
    pkcs7_unpadder = p.PKCS7(AES.block_size).unpadder()
    recovered_packet = pkcs7_unpadder.update(recovered_padded_packet) + pkcs7_unpadder.finalize()

    return recovered_packet

In [33]:
#server private key
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)

#public keys available for sensor nodes to send packets
public_key = private_key.public_key()

#packets
packets = b' sensors' #use the data of the sensors

#hybrid RSA/AES encryption of data packets
cipher_packet, cipher_key = hybrid_rsa_aes_encryption(packets, public_key)

#hybrid RSA/AES encryption of data packets
recovered_packet = hybrid_rsa_aes_decryption(cipher_packet, cipher_key, private_key)

assert (recovered_packet == packets)